In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [158]:
!wget https://www.dropbox.com/s/b6q4tokw0skjsfz/dataset.pkl?dl=0 -O dataset.pkl

--2022-09-11 20:18:24--  https://www.dropbox.com/s/b6q4tokw0skjsfz/dataset.pkl?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/b6q4tokw0skjsfz/dataset.pkl [following]
--2022-09-11 20:18:24--  https://www.dropbox.com/s/raw/b6q4tokw0skjsfz/dataset.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2e3cddfc917c56814c269987b8.dl.dropboxusercontent.com/cd/0/inline/Bszj6Q1ReW24NdMds2Dtl0ob_2ED222IjROczpu3FWQ6r_K5t9G-QTmU6B_okGjzZn0HIeetLYSsfpR92QyTC6GAcz-pGFvSlB96Gc9lCpY_RGaVfyR9438qzUQJYzzIynM4SstGwriPSY7T2xvnDaTX1m1ha0tYdvz1YwUMIMNDkQ/file# [following]
--2022-09-11 20:18:25--  https://uc2e3cddfc917c56814c269987b8.dl.dropboxusercontent.com/cd/0/inline/Bszj6Q1ReW24NdMds2Dtl0ob_2ED222IjROczpu3FWQ6r_K5t9G-QTmU6B_okGjzZn0HIeetLYSsfp

In [28]:
import pickle
with open('dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [133]:
lines = dataset['mayakovskij']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [134]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
     
# Create a reference variable for Class WordPunctTokenizer
tk = WordPunctTokenizer()
lines = [' '.join(tk.tokenize(line.lower())) for line in lines]

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [135]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    for line in lines:
      line = (UNK + ' ') * (n-1) + line + ' ' + EOS
      tokens = line.split()

      for i in range(2, len(tokens)):
        prefix = tokens[i-n+1:i]
        counts[tuple(prefix)][tokens[i]]+=1
    
    return counts


In [136]:
counts = count_ngrams(lines, 3)

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [137]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in counts.keys():
          for word in counts[prefix]:
            self.probs[prefix][word] = counts[prefix][word] / sum(counts[prefix].values())
                    
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [138]:
lm = NGramLanguageModel(lines, n=3)


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [148]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_tokens = lm.get_possible_next_tokens(prefix)
    next_probs = {next: (lm.get_next_token_prob(prefix, next) / sum(next_tokens.values()) )** (1/temperature) for next in next_tokens}
    next_token = np.random.choice(sorted(next_probs.keys(), key=next_probs.get, reverse= True))
    return next_token

In [144]:
prefix = '' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

 крестьянское хозяйство улучшит грамотей , по - утиному ! отбивает поклоны . хлоп да хлоп ! шишек десять набила на лоб . только тот коммунист истый , кто - то !) электрический ходит в поле , вдруг оглушительное : " довольно дураков ". пусть писатели начинают . подожду . посмотрю , какою дрянью заначиняют чемоданы душ . только колышутся спадающие на плечи мягкие складки лоснящихся щек . оказалось — записка от мужа . « боевые вы подружки !» — посмотрите в работе года . вот вы , женщина , на лето ведет поворот . наш вождь – миллионноглавый третий интернационал


In [149]:
prefix = 'Послушайте' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

ValueError: ignored

Ошибка, которую следовало ожидать, поскольку какое-то слово не встретилось в датасете и все сломалось)))

### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [141]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    
    log_perp = 0
    total_len = sum([len(line) for line in lines])
    for line in lines:
      perp = 0
      for i in range(len(line)):
        prefix = line.split()
        prefix = prefix[:i]
        if np.log(lm.get_next_token_prob(line[:i], line[i])) < min_logprob:
          perp += min_logprob
        else:
          perp += np.log(lm.get_next_token_prob(line[:i], line[i]))
      log_perp += perp 
   
    return np.exp(log_perp+np.log(total_len))

Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [155]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [156]:
prefix = 'Послушайте' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

Послушайте считайте при вашей перепетуя скажут штреземаны вынужден севшую кой богатство вначале изнутри барабаня гоняться егорию одинок дешевые бессонницей голодовке пьяниц треб любом стоя эй месопотамию закованного хула канаву сел предместий натолкнувшись беззаветно форсе твердо обернулся поклоны воспаленном апельсины догоняет набились тщась попробуйте саду рабочее бутылей зачерствевший годами подняла забирай оттого штучкой тротуар кровью развея американских девятнадцатом гневно возлагаю зато зерна кряхтел молитву репей усики отчаяннейшего погоста красоте разгромыхает готовить вернутся марай электротрактор больной почище сидишь ростовщиком плуги пропорол тишки оборону войти тит отдано буржуя передышка миром зарезу шахты прождала чешет хвастать зажатые запиши радуга рек соку выездом ломали тучи борются


# neural approach

In [13]:
!git clone https://github.com/ivan-chai/pyfillet.git

Cloning into 'pyfillet'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 100 (delta 44), reused 77 (delta 26), pack-reused 0
Receiving objects: 100% (100/100), 286.69 KiB | 3.93 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [14]:
!mv pyfillet pyfillet_git

In [15]:
!cp -r pyfillet_git/src/pyfillet pyfillet

In [16]:
!pip install pymorphy2==0.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 11.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=687690799d43686a65e82786f792086b4f2edc5e5ff7b999fd8e705ee1f564ef
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [17]:
!pip install gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 46.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [18]:
import pyfillet

In [6]:
import torch
from torch import nn

In [101]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 300
        self.embedding_dim = 300
        self.num_layers = 3

        n_vocab = len(dataset)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(x, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [123]:
import torch
import pandas as pd
from collections import Counter

class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        txt_list,
        tokenizer,
    ):
        self.data = txt_list
        self.tokenizer = tokenizer
        self.uniq_words = self.get_uniq_words()
        self.embeddings = []

        for txt in txt_list:
          embeddings = tokenizer(txt)
          new_emb = []
          for i in range(len(embeddings)):
            new_emb.append(embeddings[i][1])
          self.embeddings.append(new_emb)
        

    def get_uniq_words(self):
        word_counts = Counter(self.data)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return (
            self.embeddings[index], self.embeddings[index]
        )

In [120]:
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
# from model import Model
# from dataset import Dataset

def train(dataset, model, args):
    model.train()

    dataloader = DataLoader(dataset, batch_size=args['batch_size'])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(args['max_epochs']):
        state_h, state_c = model.init_state(args['sequence_length'])

        for batch, (x, y) in enumerate(dataloader):
            print(len(x),len(y))
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [121]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

In [124]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--max-epochs', type=int, default=10)
# parser.add_argument('--batch-size', type=int, default=256)
# parser.add_argument('--sequence-length', type=int, default=4)
# args = parser.parse_args()

args = {'max_epochs': 10, 'batch_size': 1, 'sequence_length': 100}

data = Dataset(dataset['mayakovskij'], embedder)
model = Model(data)

train(data, model, args)
print(predict(data, model, text='Knock knock. Whos there?'))

76 76


TypeError: ignored